In [5]:
import ipywidgets as widgets
from pathlib import Path
import json

# Define the predefined regions
PREDEFINED_REGIONS = {
    "Mangalore": {
        "min_lat": 12.8,
        "max_lat": 13.0,
        "min_lon": 74.7,
        "max_lon": 74.9
    },
    "Lakshadweep": {
        "min_lat": 10.0,
        "max_lat": 11.0,
        "min_lon": 72.0,
        "max_lon": 73.0
    },
    "Gulf of Mannar": {
        "min_lat": 9.0,
        "max_lat": 9.5,
        "min_lon": 79.0,
        "max_lon": 79.5
    }
}

# Create dropdown for region selection
region_dropdown = widgets.Dropdown(
    options=list(PREDEFINED_REGIONS.keys()),
    description='Region:',
    style={'description_width': 'initial'}
)

# Create confirm button
confirm_button = widgets.Button(
    description='Confirm Selection',
    button_style='success',
    icon='check'
)

# Create status output
status_output = widgets.Output()

# Define button click handler
def on_confirm_clicked(b):
    with status_output:
        status_output.clear_output()
        
        # Get selected region coordinates
        region_name = region_dropdown.value
        coords = PREDEFINED_REGIONS[region_name]
        
        # Create config
        config = {
            "region_name": region_name,
            "aoi": coords
        }
        
        # Save to config file
        config_dir = Path('sdb_project/config')
        config_dir.mkdir(parents=True, exist_ok=True)
        config_path = config_dir / 'location_config.json'
        
        with open(config_path, 'w') as f:
            json.dump(config, f, indent=2)
            
        print(f"✅ Saved {region_name} coordinates to {config_path}")
        print("\nSelected coordinates:")
        print(f"Latitude:  {coords['min_lat']}° to {coords['max_lat']}°")
        print(f"Longitude: {coords['min_lon']}° to {coords['max_lon']}°")

# Connect handler
confirm_button.on_click(on_confirm_clicked)

# Display widgets
widgets.VBox([
    widgets.HTML("<h3>Quick Region Selection</h3>"),
    widgets.HBox([region_dropdown, confirm_button]), 
    status_output
])

# Select Study Area Location

This notebook helps you select and configure the study area for the Satellite-Derived Bathymetry (SDB) analysis. You have two options:

## Option 1: Quick Selection

Use the dropdown menu below to select from predefined study areas. The coordinates will be automatically saved to the configuration file.

## Option 2: Custom Selection

If you need a different area, you can:
1. Enter custom coordinates directly using the form below
2. Click "View on Map" to verify the selected region
3. Click "Save Configuration" to store the coordinates

The selected area coordinates will be saved to `sdb_project/config/location_config.json` and will be used by the SDB pipeline.

In [6]:
# Import required libraries
import os
from pathlib import Path
import json
import folium
from folium import plugins
from IPython.display import display
import ipywidgets as widgets
from shapely.geometry import box
import geopandas as gpd

# Setup project directories
PROJECT_DIR = Path().absolute()
CONFIG_DIR = PROJECT_DIR / 'sdb_project' / 'config'
CONFIG_DIR.mkdir(parents=True, exist_ok=True)

print(f"Project directory: {PROJECT_DIR}")
print(f"Config directory: {CONFIG_DIR}")

Project directory: d:\Project
Config directory: d:\Project\sdb_project\config


## Important: Enable Map Display

You should see a notification bar at the top of the notebook saying "This notebook is not trusted...". To see the interactive map:

1. Click the "Trust" button in the notification bar at the top of VS Code
2. If you don't see the notification bar, try:
   - Closing and reopening the notebook
   - Or clicking the "Select Kernel" button in the top right
   - Or running "Developer: Reload Window" from the Command Palette (Ctrl+Shift+P)

This is a security feature that prevents untrusted notebooks from displaying HTML content.

## Interactive Location Selection

Use the form below to input region details. Default values are set for Lakshadweep Islands as an example.

In [ ]:
# Test direct map display
test_map = folium.Map(
    location=[10.0, 76.0],
    zoom_start=5,
    tiles='OpenStreetMap'
)

# Add a marker
folium.Marker(
    [10.0, 76.0],
    popup='Test Marker'
).add_to(test_map)

# Display map
display(test_map)

In [7]:
# Create interactive widgets for location input
region_name = widgets.Text(
    value='Lakshadweep',
    description='Region:',
    style={'description_width': 'initial'}
)

lat_min = widgets.FloatText(
    value=10.75,
    description='Min Lat:',
    step=0.01,
    style={'description_width': 'initial'}
)

lat_max = widgets.FloatText(
    value=10.95,
    description='Max Lat:',
    step=0.01,
    style={'description_width': 'initial'}
)

lon_min = widgets.FloatText(
    value=72.35,
    description='Min Lon:',
    step=0.01,
    style={'description_width': 'initial'}
)

lon_max = widgets.FloatText(
    value=72.65,
    description='Max Lon:',
    step=0.01,
    style={'description_width': 'initial'}
)

# Create form layout
form = widgets.VBox([
    widgets.HTML("<h3>Input Location Details</h3>"),
    region_name,
    widgets.HBox([lat_min, lat_max]),
    widgets.HBox([lon_min, lon_max])
])

display(form)

In [ ]:
# Define functions for map creation and config saving
def create_aoi_map():
    """Create an interactive map showing the Area of Interest"""
    # Create center point for map
    center_lat = (lat_min.value + lat_max.value) / 2
    center_lon = (lon_min.value + lon_max.value) / 2
    
    # Initialize map with a simple style
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=11,
        tiles='OpenStreetMap'
    )
    
    # Create rectangle for AOI
    bounds = [[lat_min.value, lon_min.value],
              [lat_max.value, lon_max.value]]
    
    # Add the AOI rectangle
    folium.Rectangle(
        bounds=bounds,
        color='red',
        fill=True,
        popup=f"AOI: {region_name.value}"
    ).add_to(m)
    
    # Save and display map directly
    map_file = Path('temp/aoi_map.html')
    map_file.parent.mkdir(exist_ok=True)
    m.save(str(map_file))
    
    # Display using direct HTML
    from IPython.display import HTML
    with open(map_file) as f:
        html = f.read()
    return HTML(f'<div style="width:100%;height:500px">{html}</div>')

def save_config():
    """Save location configuration to JSON file"""
    config = {
        "region_name": region_name.value,
        "aoi": {
            "min_lat": lat_min.value,
            "max_lat": lat_max.value,
            "min_lon": lon_min.value,
            "max_lon": lon_max.value
        }
    }
    
    config_path = CONFIG_DIR / 'location_config.json'
    with open(config_path, 'w') as f:
        json.dump(config, f, indent=2)
        
    print(f"\nConfiguration saved to: {config_path}")
    print("\nSaved values:")
    print(json.dumps(config, indent=2))
    
    return config

# Create buttons for actions
map_button = widgets.Button(
    description='Show Map',
    button_style='info',
    icon='map'
)

save_button = widgets.Button(
    description='Save Configuration',
    button_style='success',
    icon='save'
)

def on_map_button_clicked(b):
    display(create_aoi_map())
    
def on_save_button_clicked(b):
    save_config()

# Connect button events
map_button.on_click(on_map_button_clicked)
save_button.on_click(on_save_button_clicked)

# Display buttons
display(widgets.HBox([map_button, save_button]))

## Verify Configuration

After saving the configuration, you can verify its contents by reading the JSON file:

In [9]:
def read_config():
    """Read and display the saved configuration"""
    config_path = CONFIG_DIR / 'location_config.json'
    if config_path.exists():
        with open(config_path, 'r') as f:
            config = json.load(f)
        print("\nCurrent configuration:")
        print(json.dumps(config, indent=2))
    else:
        print("\nNo configuration file found at:", config_path)

# Add a verification button
verify_button = widgets.Button(
    description='Verify Config',
    button_style='warning',
    icon='check'
)

verify_button.on_click(lambda b: read_config())
display(verify_button)

Button(button_style='warning', description='Verify Config', icon='check', style=ButtonStyle())